In [26]:
import os
import glob
import pandas as pd 
import zipfile
import geopandas as gpd
import geoparquet as gpq
from utils import *
from congestion_metrics import *
import logging
# from simpledbf import Dbf5s

In [27]:
key_cols = ['ID', 'INODE', 'JNODE','LENGTH', 'TYPE', 'MODES', 'LANES', 'NODE_ID',
            'MSM2018','Sector_3', 'Link_Type','Description', 'Group_1', 'Group_2', 'Display'] 
baseline_cols = ['VOLAX_26116', 'VOLAU_26116', 'TIMAU_26116', 'vcv_26116', 'VOLAX_26316', 'VOLAU_26316', 'TIMAU_26316', 'vcv_26316']

CONGESTED_MAP = {11: 0.82, 12: 0.82, 13: 0.82, 14: 0.82, 15: 0.82, 16: 0.82, 17: 0.82, 
                 18: 0.9, 19: 0.9, 20: 0.9, 21: 0.9,
                 22: 0.46,
                 23: 0.58, 24: 0.58, 25: 0.58, 26: 0.58, 27: 0.58}
MIN_TYPE = 1

suffix = 'p1'
# suffix = 'p2_1'

if suffix == 'p1':
    baseline_scenario_list = [50,74]
    zip_path = '../inputs/emme_links/phase_1/p10_Shapefiles.zip'
    zip_paths = ['../inputs/emme_links/phase_1/p10_Shapefiles.zip', '../inputs/emme_links/phase_1/16_TOUCH_base_AGSv1_E23B_NC.zip',
                 '../inputs/emme_links/phase_1/76_TOUCH_Base_3Ca.zip', '../inputs/emme_links/phase_1/52_TOUCH_Option_1B.zip',
                 '../inputs/emme_links/phase_1/56_TOUCH_Option_2A.zip', '../inputs/emme_links/phase_1/58_TOUCH_Option_2B.zip',
                 '../inputs/emme_links/phase_1/64_TOUCH_Option_3A.zip', '../inputs/emme_links/phase_1/70_TOUCH_Option_3D.zip',
                 '../inputs/emme_links/phase_1/80_TOUCH_Option_3F.zip', '../inputs/emme_links/phase_1/82_TOUCH_Option_3G.zip']
    SCEN_MAP = SCENARIO_P1_MAP.copy()
    
elif suffix == 'p2_1':
    baseline_scenario_list = [16,18]
    zip_path = '../inputs/emme_links/007_Shapefiles.zip'
    zip_paths = ['../inputs/emme_links/007_Shapefiles.zip', '../inputs/emme_links/22_TOUCH_Option_1A2.zip', '../inputs/emme_links/82_TOUCH_Option_3H2_P2_1.zip']
    SCEN_MAP = SCENARIO_P2_1_MAP.copy()


'''

In [28]:
ROAD_LINKS_BASE_COLS = ['ID', 'INODE', 'JNODE', 'LENGTH', 'TYPE', 'MODES', 'LANES', '@v0', 'geometry']
ROAD_LINKS_VOL_COLS =  ['ID', 'VOLAX', 'VOLAU', 'VOLAD', 'TIMAU', '@vcv', 
                        '@eb7', '@eb7p', '@hbw7', '@hbw7p', '@hbsh7', '@hbsh7p', '@hcv7', '@toll']
PT_LINKS_VOL_COLS = ['ID', 'VOLTR', 'TIMTR']

## read nodes and links
def year(scen_code):
    if scen_code == 12:
        return 18
    else:
        return YEAR

def read_linkchar_from_zip(zip_path, baseline_scenario = [16,18]):

    temp_dfs = []
    speed_dfs = []
    speed_counter = 0

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()

    for scenario in baseline_scenario:
        for period_no in (PERIOD_MAP.values()):
            scenario_no = "26" + str(period_no) + str(scenario)
            file_list = zip_ref.namelist()
            for file in file_list:
                print(file)
                if (scenario_no in file) and (file.endswith('emme_links.shp')):
                    temp_df = gpd.read_file(f"zip://{zip_path}!{file}")[ROAD_LINKS_BASE_COLS]# .rename(columns={'v0':'FF_SPEED'})
                    temp_df = temp_df[temp_df['TYPE']>=MIN_TYPE]

                    temp_speed_df = temp_df[['ID', '@v0']].rename(columns={'@v0': f'FFSPEED_{period_no}'})
                    temp_df = temp_df.drop(columns='@v0')
                    temp_dfs.append(temp_df)

                    if speed_counter == 0:
                        speed_df = temp_speed_df.copy()
                    else:
                        speed_df = speed_df.merge(temp_speed_df, on ='ID', how='left')
                    speed_counter += 1
        
        speed_counter = 0
        speed_dfs.append(speed_df)

    linkchar_df = pd.concat(temp_dfs).drop_duplicates(subset=['ID', 'INODE', 'JNODE'])
    linkspeed_df = pd.concat(speed_dfs).drop_duplicates(subset=['ID'])

    linkchar_df = linkchar_df.merge(linkspeed_df, on='ID', how='left')               

    return linkchar_df

# msm_links_dfs, msm_nodes_df = read_nodes_and_links()
linkchar_df = read_linkchar_from_zip(zip_path, baseline_scenario = baseline_scenario_list)
# read_links_from_zip(zip_path)

50_TOUCH_Option_1A/Scenario_26150/emme_links.cpg
50_TOUCH_Option_1A/Scenario_26150/emme_links.dbf
50_TOUCH_Option_1A/Scenario_26150/emme_links.prj
50_TOUCH_Option_1A/Scenario_26150/emme_links.shp
50_TOUCH_Option_1A/Scenario_26150/emme_links.shx
50_TOUCH_Option_1A/Scenario_26150/emme_nodes.cpg
50_TOUCH_Option_1A/Scenario_26150/emme_nodes.dbf
50_TOUCH_Option_1A/Scenario_26150/emme_nodes.prj
50_TOUCH_Option_1A/Scenario_26150/emme_nodes.shp
50_TOUCH_Option_1A/Scenario_26150/emme_nodes.shx
50_TOUCH_Option_1A/Scenario_26150/emme_tlines.cpg
50_TOUCH_Option_1A/Scenario_26150/emme_tlines.dbf
50_TOUCH_Option_1A/Scenario_26150/emme_tlines.prj
50_TOUCH_Option_1A/Scenario_26150/emme_tlines.shp
50_TOUCH_Option_1A/Scenario_26150/emme_tlines.shx
50_TOUCH_Option_1A/Scenario_26150/emme_tsegs.cpg
50_TOUCH_Option_1A/Scenario_26150/emme_tsegs.dbf
50_TOUCH_Option_1A/Scenario_26150/emme_tsegs.prj
50_TOUCH_Option_1A/Scenario_26150/emme_tsegs.shp
50_TOUCH_Option_1A/Scenario_26150/emme_tsegs.shx
50_TOUCH_Option

In [29]:
def rename_columns(suffix, original_cols = ROAD_LINKS_VOL_COLS, exceptions = ["ID"]):
    new_cols = {}
    for col in original_cols:
        if col not in exceptions:
            new_cols[col] = col + "_" + suffix
        else: 
            new_cols[col] = col 
    return new_cols

def read_linkvols_from_zip(zip_paths):
    counter = 0 
    
    SCENARIO_P2_LIST = list(SCEN_MAP.values())

    file_list = []
    for zip_path in zip_paths:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_file_list = [f'zip://{zip_path}!{x}' for x in zip_ref.namelist()]
            file_list += zip_file_list

    file_dict = {item.split("/")[-2].split("_")[-1]: item for item in file_list if 'emme_links.shp' in item}

    for slot_no in SCENARIO_P2_LIST:
        if slot_no >=16:
            print(slot_no)
            for period_no in (PERIOD_MAP.values()):
                scenario_no = "26" + str(period_no) + str(slot_no)
                if scenario_no in file_dict:
                    temp_df = gpd.read_file(file_dict[scenario_no])[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(scenario_no, ROAD_LINKS_VOL_COLS))
                    if counter == 0:
                        linkvols_df = temp_df.copy()
                    else:
                        linkvols_df = linkvols_df.merge(temp_df, on='ID', how='left')
                    counter +=1

    return linkvols_df

linkvols_df = read_linkvols_from_zip(zip_paths)

16
50
52
54
56
58
64
66
70
72
74
76
78
80
82


In [30]:
def read_nodes_from_zip(zip_path, baseline_scenario = [16,18]):

    temp_dfs = []

    # zip_path = '../inputs/emme_links/007_Shapefiles.zip'

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()

    for scenario in baseline_scenario:
        for period_no in (PERIOD_MAP.values()):
            scenario_no = "26" + str(period_no) + str(scenario)
            for file in file_list:
                if (scenario_no in file) and (file.endswith('emme_nodes.shp')):
                    temp_df = gpd.read_file(f"zip://{zip_path}!{file}")[['ID', 'ISZONE', 'geometry']]
                    #print(temp_df)
                    temp_dfs.append(temp_df)
        
    nodes_df = pd.concat(temp_dfs).drop_duplicates(subset=['ID'])  

    return nodes_df

def map_nodes_to_zone(nodes_df):
    return nodes_df.sjoin_nearest(MSM_ZONES[['MSM2018', 'LBA_Name','geometry']].rename(columns={'LBA_Name':'LBA'}), 
                                  how='left').rename(columns={'ID': 'NODE_ID'}).drop(columns='index_right')

nodes_df = map_nodes_to_zone(read_nodes_from_zip(zip_path, baseline_scenario = baseline_scenario_list))
# nodes_df.to_file('../outputs/links/nodes.gpkg')

In [31]:
linkchar_lba_df = linkchar_df.merge(nodes_df[['NODE_ID', 'LBA']].rename(columns={'NODE_ID': 'INODE'}), on='INODE', how='left')
links_df = linkchar_lba_df.merge(linkvols_df, on='ID', how='left').fillna(0)
links_df

,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,geometry,FFSPEED_1,FFSPEED_2,...,TIMAU_26382,@vcv_26382,@eb7_26382,@eb7p_26382,@hbw7_26382,@hbw7p_26382,@hbsh7_26382,@hbsh7p_26382,@hcv7_26382,@toll_26382
0,1-10434,1,10434,8.947524,1,abw,1.0,"LINESTRING (1744722.512 5988249.515, 1735792 5...",40.0,40.0,...,19.327909,0.814491,22.374872,24.164864,75.303375,83.586746,4.424443,6.990620,8.675142,0.0
1,1-10448,1,10448,5.237727,1,abw,1.0,"LINESTRING (1744722.512 5988249.515, 1747206 5...",40.0,40.0,...,23.412083,1.186212,41.442352,44.757744,66.064026,73.331070,18.042620,28.507339,38.213120,0.0
2,2-10425,2,10425,0.674503,1,abw,1.0,"LINESTRING (1736211.418 5982441.177, 1736501 5...",40.0,40.0,...,2.230946,1.047729,29.487726,31.846746,86.094948,95.565392,12.749667,20.144474,19.293354,0.0
3,2-10427,2,10427,0.857741,1,abw,1.0,"LINESTRING (1736211.418 5982441.177, 1736562 5...",40.0,40.0,...,2.347477,0.952913,9.236068,9.974954,98.662125,109.514960,20.428671,32.277302,4.149139,0.0
4,3-10427,3,10427,2.108739,1,abw,1.0,"LINESTRING (1737358.501 5985176.528, 1736562 5...",40.0,40.0,...,3.361881,0.500681,19.062338,20.587326,33.181087,36.831009,10.010269,15.816226,6.821815,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19793,15105-13058,15105,13058,0.087201,7,r,1.0,"LINESTRING (1772305 5890125, 1772357 5890195)",60.0,60.0,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
19794,15105-15104,15105,15104,0.040224,7,r,1.0,"LINESTRING (1772305 5890125, 1772282 5890092)",60.0,60.0,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
19795,5242-7334,5242,7334,0.911615,16,a,1.0,"LINESTRING (1758186 5920245, 1757655 5919504)",55.0,55.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
19796,5340-7334,5340,7334,0.513016,16,a,1.0,"LINESTRING (1757856 5919976, 1757655 5919504)",55.0,55.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [32]:
detailed_col = [x for x in links_df.columns.tolist() if (("@" in x) and ("@vcv" not in x) and ("@toll" not in x))]
simple_col = [x for x in links_df.columns.tolist() if ((x not in detailed_col) and ("VOLAD" not in x))]

In [33]:
# links_df.to_file(f'../outputs/links/linkvols_{suffix}.gpkg')
# links_df.drop(columns='geometry').to_csv(F'../outputs/links/linkvols_{suffix}.csv', index=None)

# links_df[simple_col].to_file(f'../outputs/links/linkvolsa_{suffix}.gpkg')
# links_df[simple_col].drop(columns='geometry').to_csv(f'../outputs/links/linkvolsa_{suffix}.csv', index=None)

links_df[simple_col].to_file(f'../outputs/links/alllinkvolsa_{suffix}.gpkg')
links_df[simple_col].drop(columns='geometry').to_csv(f'../outputs/links/alllinkvolsa_{suffix}.csv', index=None)

In [34]:
# toll_cols = ['ID', 'INODE', 'JNODE'] + [x for x in links_df.columns.tolist() if (('@toll' in x))] + ['geometry']
# links_df[toll_cols].to_file(f'../outputs/links/tolled_links_{suffix}.gpkg') 

In [35]:
# pnr_zones = msm_nodes_df[msm_nodes_df['ID']>90000]
# pnr_zones.sjoin(MSM_ZONES, predicate='within')[['ID', 'MSM2018', 'LBA_Name', 'Sector_1', 'Sector_2', 'Sector_3']]# .to_csv(os.path.join(INPUT_DIR, 'PnR_Zones.csv'))